In [1]:
import pandas as pd
import numpy as np
import ta
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice
from ta.trend import EMAIndicator
from ta.trend import SMAIndicator
from ta.volatility import BollingerBands
import time
import datetime as dt
from datetime import date
from datetime import timedelta
import math
import os

In [2]:
from kiteconnect import KiteConnect
from selenium import webdriver

In [3]:
cwd = os.chdir("C:\\Users\\Nikhil\\Desktop\\Swarnim assignment")

In [4]:
def autologin():
    token_path = "cred.txt"
    key_secret = open(token_path,'r').read().split()
    kite = KiteConnect(api_key=key_secret[0])
    service = webdriver.chrome.service.Service('./chromedriver')
    service.start()
    options = webdriver.ChromeOptions()
    options.add_argument('incognito')
    options.add_argument('--headless')
    options = options.to_capabilities()
    driver = webdriver.Remote(service.service_url, options)
    driver.get(kite.login_url())
    driver.implicitly_wait(10)
    username = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[1]/input')
    password = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/input')
    username.send_keys(key_secret[2])
    password.send_keys(key_secret[3])
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[4]/button').click()
    pin = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/input')
    pin.send_keys(key_secret[4])
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[3]/button').click()
    time.sleep(10)
    request_token=driver.current_url.split("request_token=")[1].split("&action")[0]
    with open('request_token.txt', 'w') as the_file:
        the_file.write(request_token)
    driver.quit()

autologin()

#generating and storing access token - valid till 6 am the next day
request_token = open("request_token.txt",'r').read()
key_secret = open("cred.txt",'r').read().split()
kite = KiteConnect(api_key=key_secret[0])
data = kite.generate_session(request_token, api_secret=key_secret[1])
with open('access_token.txt', 'w') as file:
        file.write(data["access_token"])

In [5]:
access_token = open("access_token.txt",'r').read()
key_secret = open("cred.txt",'r').read().split()
kite = KiteConnect(api_key=key_secret[0])
kite.set_access_token(access_token)

In [7]:
instrument_dump = kite.instruments("NFO")
instrument_df = pd.DataFrame(instrument_dump)

In [8]:
equity_dump = kite.instruments("NSE")
equity_df  = pd.DataFrame(equity_dump)

In [9]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1


def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval, oi=True))
    #data.set_index("date",inplace=True)
    return data

In [10]:
def EqLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return equity_df[equity_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1


def fetchEqOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = EqLookup(equity_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval, oi=True))
    return data

In [11]:
def add_indicator(Data):
    EMA20  = EMAIndicator(close = Data["close"], window = 20, fillna = True)
    EMA8  = EMAIndicator(close = Data["close"], window = 8, fillna = True)
    SMA4Volume = SMAIndicator(close= Data["volume"], window=4, fillna= True)
    SMA20Volume = SMAIndicator(close= Data["volume"], window=20, fillna= True)
    RSI = RSIIndicator(close= Data["close"], window=14, fillna= True)
    VWAP = VolumeWeightedAveragePrice(Data['high'], Data['low'], Data['close'], Data['volume'], 14, True)
    
    
    Data['RSI']= RSI.rsi()
    Data['EMA8']= EMA8.ema_indicator()
    Data['EMA20']= EMA20.ema_indicator()
    Data['SMA4Volume']= SMA4Volume.sma_indicator()
    Data['SMA20Volume']= SMA20Volume.sma_indicator()
    
    Data['Vwap']=VWAP.volume_weighted_average_price()
    
    return Data
    

In [12]:
df=pd.read_excel('fno_stocks.xlsx', header= 1)
Tickers = df["Symbol"]
Tickers = Tickers.astype(str)

In [22]:
log = pd.DataFrame(columns= ["Ticker", "per_change"])

for i in Tickers:
    if(i== 'NIFTY'or i=='BANKNIFTY'):
        continue
    daily_data=fetchEqOHLC(i,'day',5)
    data=fetchEqOHLC(i,'15minute',1)
    per_change=((data['close'][daily_data.shape[0]-1]-daily_data['close'][daily_data.shape[0]-2])/daily_data['close'][daily_data.shape[0]-2])
    new_row = {"Ticker":i, "per_change": per_change}
    log = log.append(new_row, ignore_index = True)

In [23]:
log.sort_values(by=['per_change'],ascending=False,inplace=True)

In [24]:
log.head(10)

,Ticker,per_change
154,POLYCAB,0.031061
135,IPCALAB,0.027684
61,MINDTREE,0.022073
120,COALINDIA,0.020518
97,IRCTC,0.018816
134,LTTS,0.018724
19,NATIONALUM,0.017356
99,ABFRL,0.016791
20,HINDALCO,0.014777
167,APOLLOHOSP,0.014535


In [25]:
# If you want to remove condition write true inside -- if(True)
def check_intraday_long_conditions(Data,j,Data3):
    if((Data3['close'][j]-Data['close'][j] > 0.0025*Data['close'][j] or Data['close'][j]-Data['open'][j]>0.0025*Data['close'][j]) and Data['close'][j-1]-Data['open'][j-1] < 0.00125*Data['close'][j-1] ):
        if((Data3['volume'][j]>Data['SMA4Volume'][j] or Data['volume'][j]>Data['SMA4Volume'][j]) and Data['close'][j]> Data['EMA20'][j] and Data['close'][j-2]-Data['open'][j-2] < 0.00125*Data['close'][j-2]):
            if(Data['SMA4Volume'][j-1]< Data['SMA20Volume'][j-1]):
                return True
                        
    return False

In [26]:
while(True):
    flag=0
    for i in log['Ticker'][:5]:
        data=fetchEqOHLC(i,'5minute',2)
        data3=fetchEqOHLC(i,'3minute',2)
        data=add_indicator(data)
        if(check_intraday_long_conditions(data,data.shape[0]-1,data3)==True):
            print(str(i)+ '   '+ str(dt.datetime.now().time()))
            
            flag=flag+1
    time.sleep(30) 

POLYCAB   12:54:04.170108
IPCALAB   12:54:05.010632
POLYCAB   12:54:37.470424
IPCALAB   12:54:37.990828
IPCALAB   12:55:45.150746
IPCALAB   12:56:18.467380
IPCALAB   12:56:51.460181
IPCALAB   12:57:25.030414
IPCALAB   12:57:58.405661


KeyboardInterrupt: 